In [5]:
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import os
%matplotlib inline

In [12]:
NOTEBOOK_DIR = os.getcwd()
BASE_DIR = os.path.dirname(NOTEBOOK_DIR)
DATA_DIR = os.path.join(BASE_DIR, "data")
DATA_DIR_PATH = os.path.join(DATA_DIR, "Data_cars.csv")
DATA_ZIP_DIR_PATH = os.path.join(DATA_DIR, "Data_cars.csv.zip")

In [14]:
if not os.path.exists(DATA_DIR_PATH):
    !unzip "${DATA_ZIP_DIR_PATH}"

In [66]:
data_car = pd.read_csv(DATA_DIR_PATH)

In [16]:
data_car.head()

,Price,Make,Model,Model_year,Mileage,Fuel,Gearbox,Online,Description
0,11220.0,PEUGEOT,308,2014,94341.0 km,Diesel,mécanique,06/07/2018 à 3h47,"modele: 308 (2E GENERATION), version: 1.6 HDI ..."
1,57526.0,BMW,X6,2015,39051.0 km,Diesel,automatique,03/04/2018 à 16h41,"modele: X6 F16, version: (F16) XDRIVE30D 258 M..."
2,80379.0,AUDI,RS6,2014,75381.0 km,Essence,automatique,30/07/2018 à 1h55,"modele: RS6 (3E GENERATION) AVANT, version: II..."
3,2830.0,FORD,KA,2007,92282.0 km,Essence,mécanique,09/07/2018 à 14h12,"modele: KA, version: 1.3 70 FUN, puissance_fis..."
4,12925.0,FIAT,TIPO,2018,10.0 km,Essence,mécanique,19/05/2018 à 3h52,"modele: TIPO 2 SW, version: II SW 1.4 95 POP, ..."


In [23]:
data_car.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 166695 entries, 0 to 166694
Data columns (total 9 columns):
Price          166695 non-null float64
Make           166695 non-null object
Model          166695 non-null object
Model_year     166695 non-null int64
Mileage        166695 non-null object
Fuel           166695 non-null object
Gearbox        166695 non-null object
Online         166695 non-null object
Description    166695 non-null object
dtypes: float64(1), int64(1), object(7)
memory usage: 11.4+ MB


In [41]:
data_car.Online.values[80:90]

array(['10/06/2018 à 9h14', '31/03/2018 à 4h20', '07/06/2018 à 20h33',
       '16/01/2018 à 11h58', '11/05/2018 à 20h34', '23/03/2018 à 19h50',
       '23/04/2018 à 9h50', '20/03/2018 à 5h12', '13/05/2018 à 18h46',
       '14/04/2018 à 4h22'], dtype=object)

In [43]:
from datetime import datetime
def online_clean(df):
    datetime_format = "%d/%m/%Y à %Hh%M"
    df.Online = [datetime.strptime(d, datetime_format) for d in df.Online.values]
    return df

In [44]:
def mileage_clean(df):
    df.Mileage = [float(m.split(' ')[0]) for m in df.Mileage.values]
    return df

In [56]:
import re
regex_pattern = r"modele:\s*(?P<modele>.*?(?=,)),\sversion:\s*(?P<version>.*?(?=,)),\spuissance_fiscale:\s*(?P<puissance_fiscale>.*?(?=,)),\sportes:\s*(?P<portes>.*?(?=,)),\soptions:\s*(?P<Descriptions>.*?(?=,)),\scouleur:\s(?P<couleur>.*$)"
match = re.search(regex_pattern, data_car_preprocessed.Description[12])
print("Match at index %s, %s" % (match.start(), match.end()))
print("Full match: %s" % (match.group(0)))
print("modele: %s" % (match.group(1)))
print("version: %s" % (match.group(2)))
print("spuissance_fiscale: %s" % (match.group(3)))
print("portes: %s" % (match.group(4)))
print("options: %s" % (match.group(5)))
print("couleur: %s" % (match.group(6)))

Match at index 0, 1705
Full match: modele: 500 X, version: 1.4 MULTIAIR 140 LOUNGE 4X2 DCT, puissance_fiscale: 7, portes: 5.0, options: écran tactile;palettes au volant;volant cuir;régulateur limiteur de vitesse;prises audio auxiliaires;cache bagages;sorties d'échappement chromées;direction assistée;rétroviseurs électriques;airbag frontal;banquette 1/3 - 2/3;vitres ar. surteintées;appui-tête conducteur réglable hauteur;commandes vocales;GPS;prise audio USB;commande du comportement dynamique;régulateur de vitesse;projecteurs xénon;carte main libre;jantes alu;boucliers av et ar couleur caisse;phares av. de jour;airbags rideaux;projecteurs antibrouillard;climatisation automatique multi zone;vitres électriques;eclairage d'ambiance;appui-tête passager réglable en hauteur;rétroviseurs rabattables électriquement;ecran multifonction couleur;siège passager rabattable;aide parking;kit de réparation crevaison;contrôle de pression des pneus;airbags front. + lat.;essuie-glaces automatiques;alerte f

In [69]:
def descriptions_clean(df):
    regex_pattern = r"modele:\s*(?P<modele>.*?(?=,)),\sversion:\s*(?P<version>.*?(?=,)),\spuissance_fiscale:\s*(?P<puissance_fiscale>.*?(?=,)),\sportes:\s*(?P<portes>.*?(?=,)),\soptions:\s*(?P<Descriptions>.*?(?=,)),\scouleur:\s(?P<couleur>.*$)"
    version = []
    spuissance_fiscale = []
    portes = []
    options = []
    couleur = []
    for i in range(df.shape[0]):
        match = re.search(regex_pattern, data_car_preprocessed.Description[i])
        version.append(match.group(2))
        spuissance_fiscale.append(match.group(3))
        portes.append(match.group(4))
        options.append(match.group(5))
        couleur.append(match.group(6))
    df["version"] = list(map(str.lower, version)
    df["spuissance_fiscale"] = spuissance_fiscale
    df["portes"] = portes
    df["options"] = options
    df["couleur"] = list(map(str.lower, couleur)
    del df["Description"]
    return df

In [65]:
def preprocessing(data, preprocessors=None):
    df = data.copy(deep=True)
    if preprocessors is not None:
        for preprocessor in preprocessors:
            df = preprocessor(df)
        return df
    else:
        return df

In [70]:
data_car_preprocessed = preprocessing(data_car, [online_clean, mileage_clean, descriptions_clean])

In [71]:
data_car_preprocessed.head()

,Price,Make,Model,Model_year,Mileage,Fuel,Gearbox,Online,version,spuissance_fiscale,portes,options,couleur
0,11220.0,PEUGEOT,308,2014,94341.0,Diesel,mécanique,2018-07-06 03:47:00,1.6 HDI FAP 92CH BUSINESS 5P,4,5.0,banquette arrière 3 places;volant cuir;cache b...,BLANC BANQUISE
1,57526.0,BMW,X6,2015,39051.0,Diesel,automatique,2018-04-03 16:41:00,(F16) XDRIVE30D 258 M SPORT BVA8,16,5.0,triangle de presignalisation et trousse de pre...,Carbonschwarz metallise
2,80379.0,AUDI,RS6,2014,75381.0,Essence,automatique,2018-07-30 01:55:00,III AVANT 4.0 TFSI 560 QUATTRO TIPTRONIC,47,5.0,banquette arrière 3 places;écran tactile;témoi...,NOIR PANTHERE CRISTAL
3,2830.0,FORD,KA,2007,92282.0,Essence,mécanique,2018-07-09 14:12:00,1.3 70 FUN,5,3.0,direction assistée;jantes alu;rétroviseurs éle...,JAUNE
4,12925.0,FIAT,TIPO,2018,10.0,Essence,mécanique,2018-05-19 03:52:00,II SW 1.4 95 POP,5,5.0,Volant alu & cuir;Bluetooth inclut musique en ...,blanc verni


In [72]:
data_car_preprocessed.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 166695 entries, 0 to 166694
Data columns (total 13 columns):
Price                 166695 non-null float64
Make                  166695 non-null object
Model                 166695 non-null object
Model_year            166695 non-null int64
Mileage               166695 non-null float64
Fuel                  166695 non-null object
Gearbox               166695 non-null object
Online                166695 non-null datetime64[ns]
version               166695 non-null object
spuissance_fiscale    166695 non-null object
portes                166695 non-null object
options               166695 non-null object
couleur               166695 non-null object
dtypes: datetime64[ns](1), float64(2), int64(1), object(9)
memory usage: 16.5+ MB


In [80]:
data_car_preprocessed.options.nunique()

116129

In [ ]:
years = mdates.YearLocator()   # every year
months = mdates.MonthLocator()  # every month
years_fmt = mdates.DateFormatter('%Y')

fig, ax = plt.subplots()
ax.plot('date', 'prices', data=data)

# format the ticks
ax.xaxis.set_major_locator(years)
ax.xaxis.set_major_formatter(years_fmt)
ax.xaxis.set_minor_locator(months)

# round to nearest years.
datemin = np.datetime64(data['date'][0], 'Y')
datemax = np.datetime64(data['date'][-1], 'Y') + np.timedelta64(1, 'Y')
ax.set_xlim(datemin, datemax)

# format the coords message box
ax.format_xdata = mdates.DateFormatter('%Y-%m-%d')
ax.format_ydata = lambda x: '$%1.2f' % x  # format the price.
ax.grid(True)

# rotates and right aligns the x labels, and moves the bottom of the
# axes up to make room for them
fig.autofmt_xdate()

plt.show()